In [5]:
from pandas_datareader import data
import datetime

start = datetime.datetime(2016, 1, 1)
end = datetime.datetime(2016, 12, 1 )

sp500 = data.DataReader("^GSPC", 'yahoo', start, end)

sp500.loc[:, 'avg_price'] = (sp500.loc[:, 'High'] + sp500.loc[:, 'Low'] + sp500.loc[:, 'Close'])/3

sp500.head()

,Open,High,Low,Close,Volume,Adj Close,avg_price
Date,,,,,,,
2016-01-04,2038.199951,2038.199951,1989.680054,2012.660034,4304880000,2012.660034,2013.513346
2016-01-05,2013.780029,2021.939941,2004.170044,2016.709961,3706620000,2016.709961,2014.273315
2016-01-06,2011.709961,2011.709961,1979.050049,1990.260010,4336660000,1990.260010,1993.673340
2016-01-07,1985.319946,1985.319946,1938.829956,1943.089966,5076590000,1943.089966,1955.746623
2016-01-08,1945.969971,1960.400024,1918.459961,1922.030029,4664940000,1922.030029,1933.630005


In [6]:
import pandas as pd

it_stock = pd.read_csv('/Users/Maxwell/PycharmProjects/Github/FinancialProgramClass/companylist_it.csv').loc[:, 'Symbol']
bank_stock = pd.read_csv('/Users/Maxwell/PycharmProjects/Github/FinancialProgramClass/companylist_bank.csv').loc[:, 'Symbol']
df = pd.DataFrame()
for ticker in it_stock:
    
    df_temp = data.DataReader(ticker, 'yahoo', start, end)
    df_temp.loc[:, 'Ticker'] = str(ticker)
    df_temp.loc[:, 'Industry'] = 'IT'
    df = pd.concat([df, df_temp])

for ticker in bank_stock:
    
    df_temp = data.DataReader(ticker, 'yahoo', start, end)
    df_temp.loc[:, 'Ticker'] = str(ticker)
    df_temp.loc[:, 'Industry'] = 'Bank'
    df = pd.concat([df, df_temp])

df.loc[:, 'avg_price'] = (df.loc[:, 'High'] + df.loc[:, 'Low'] + df.loc[:, 'Close'])/3

In [45]:
def arithmetic_return_action(df, k = 30, p = 0.05, t = 0.15):
    '''
    This function will calculate arithmetic return of k days and based on the sum return to determince actions
    
    :param df: stock data
    :param k: number of days
    :param p: cutoff rate
    :return: stock data labeled with actions
    '''

    import numpy as np
    df.loc[:, 'temp_return'] = 0
    df.loc[:, 'sum_return']  = 0
    
    
    for i in range(1, k + 1):
        
        df.loc[:, 'temp_return'] = (df.loc[:, 'avg_price'].shift(i) - df.loc[:, 'Close']) / df.loc[:, 'Close']
        df.loc[np.fabs(df.temp_return) > p, 'sum_return'] += df.loc[(np.fabs(df.temp_return) > p), 'temp_return']

    
    df.loc[df.sum_return >= t, 'action'] = 'buy'    
    df.loc[(df.sum_return < t) & (df.sum_return > -t), 'action'] = 'hold'  
    df.loc[df.sum_return < -t, 'action'] = 'sell' 
    
    return df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume', 'avg_price', 'sum_return', 'action']]

In [50]:
sp500_action = arithmetic_return_action(sp500)

sp_3m = sp500_action.loc[:'2016-3-30']

sp_3m.describe()

,Open,High,Low,Close,Volume,avg_price,sum_return
count,60.000000,60.000000,60.000000,60.000000,6.000000e+01,60.000000,60.000000
mean,1949.065826,1962.077657,1932.394336,1949.416329,4.573058e+09,1947.962774,-0.200921
std,62.592920,59.399897,67.653224,64.209979,7.304275e+08,63.404883,0.451638
min,1833.400024,1847.000000,1810.099976,1829.079956,2.809090e+09,1828.726644,-1.187672
25%,1901.590027,1916.912506,1877.042511,1901.167511,4.108312e+09,1897.383341,-0.590312
50%,1937.014954,1948.854980,1919.869995,1939.030029,4.579425e+09,1934.628336,0.000000
75%,2000.587494,2012.767456,1989.455017,2004.485016,5.027802e+09,2002.406667,0.131529
max,2058.270020,2072.209961,2058.270020,2063.949951,6.503140e+09,2064.809977,0.586153
